# Assignment - Segmenting and Clustering Neighborhoods in Toronto

name: David Lu

* [Part 1: Load Toronto Neighborhoods into DataFrame by Postal Code](#part-1:-load-toronto-neighborhoods-into-dataframe-by-postal-code)
* [Part 2: Get Geo-Coordinates of Toronto Neighborhoods](#part-1:-load-toronto-neighborhoods-into-dataframe-by-postal-code)
* [Part 3: Explore Toronto Neighborhoods](#part-1:-load-toronto-neighborhoods-into-dataframe-by-postal-code)


In [1]:
import pandas as pd
from geopy.geocoders import Nominatim

## Part 1: Load Toronto Neighborhoods into DataFrame by Postal Code

I have included the markdowns indicating the validation steps taken to make sure the DataFrame consists of data with specified format.

In [2]:
tables = pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')
print(f'Total tables: {len(tables)}')

Total tables: 3


In [3]:
df = tables[0]
df.rename(
    columns = {'Postal Code': 'PostalCode', 'Borough': 'Borough', 'Neighbourhood': 'Neighborhood'},
    inplace = True)
print(df.shape)
df.head()

(180, 3)


,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


### Checking DataFrame is Created Correctly

#### The dataframe consist of three columns: PostalCode, Borough, and Neighborhood

In [4]:
assert df.columns.to_list() == ['PostalCode', 'Borough', 'Neighborhood']

#### Remove all entries where Borough is "Not assigned"

In [5]:
num_notassigned_in_borough = (df.Borough == 'Not assigned').sum()
print(f"There are {num_notassigned_in_borough} 'Not assigned' in Borough")

There are 77 'Not assigned' in Borough


In [6]:
df_ = df[df.Borough != 'Not assigned'].reset_index(drop=True)
print(df_.shape)
df_.head(12)

(103, 3)


,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
5,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
6,M1B,Scarborough,"Malvern, Rouge"
7,M3B,North York,Don Mills
8,M4B,East York,"Parkview Hill, Woodbine Gardens"
9,M5B,Downtown Toronto,"Garden District, Ryerson"


### Make sure there are no repeated Postal Code

In [7]:
if df_.PostalCode.nunique() == df_.shape[0]:
    print("There are no repeated Postal Code")
else:
    print("There are repeated Postal Code, and we must clean them such that they're listed in Neighborhood")

There are no repeated Postal Code


### Check if there are any borough but Not assigned neighborhood

In [8]:
if not (df_['Neighborhood'] == 'Not assigned').values.all():
    print(f"There are no 'Not assigned' value in the 'Neighborhood' column of the DataFrame.")
else:
    print(f"Must clean the DataFrame accordingly")

There are no 'Not assigned' value in the 'Neighborhood' column of the DataFrame.


### Print out the shape of the DataFrame

In [9]:
print(f"The shape of the DataFrame is {df_.shape}.")

The shape of the DataFrame is (103, 3).


## Part 2: Get Geo-Coordinates of Toronto Neighborhoods

In this section, the CSV file was used to create the DataFrame. 

In [10]:
data = './data/Geospatial_Coordinates.csv'
df_geo = pd.read_csv(data)
df_geo.rename(
    columns = {'Postal Code': 'PostalCode'},
    inplace = True)
df_geo.head()

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [11]:
tmp = df_.set_index('PostalCode').join(df_geo.set_index('PostalCode')).reset_index()
tmp.head(12)

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
5,M9A,Etobicoke,"Islington Avenue, Humber Valley Village",43.667856,-79.532242
6,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
7,M3B,North York,Don Mills,43.745906,-79.352188
8,M4B,East York,"Parkview Hill, Woodbine Gardens",43.706397,-79.309937
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937


## Part 3: Explore Toronto Neighborhoods

The same exploration procedures implemented in the Course Lab - "Segmenting and Clustering Neighborhoods in New York City" are performed here. The Foursquare API was used to obtain the venues data in Toronto Neighborhoods. As suggested by the assignment instruction, the data explorations was performed only on Boroughs with "Toronto" in it.

Note:
I've noticed that the Venue Category extracted consist of the value 'Neighborhood', which contradicts to the actual column name 'Neighborhood' when performing One Hot encoding. Thus, I named the column 'Neighborhood Title' at that step, and rename it back to 'Neighborhood' after we obtain the top 10 venues.

In [12]:
toronto = tmp[tmp.Borough.isin(['Downtown Toronto', 'East Toronto', 'West Toronto', 'Central Toronto'])].reset_index(drop = True)

print(toronto.shape)
toronto.head()

(39, 5)


,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
3,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
4,M4E,East Toronto,The Beaches,43.676357,-79.293031


In [13]:
address = 'Toronto, ON'
geolocator = Nominatim(user_agent="on_explorer")
location = geolocator.geocode(address)

In [14]:
import folium
neighborhoods = toronto
latitude, longitude = location.latitude, location.longitude

# create map of New York using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], neighborhoods['Borough'], neighborhoods['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [15]:
CLIENT_ID = '2P0SWWQDIBFLA5CAKFSN00DTUMGCDXS5HDODYCPDGF5PKAOU' # your Foursquare ID
CLIENT_SECRET = '0F45RJR2B5A12QM4D1RBPGMPJPQ4MLIMKUFGWDAIPIWS4YSB' # your Foursquare Secret
ACCESS_TOKEN = 'SVDUMDMBGRBCRBW00I41OVQOGT4EIX5X1W4SX4UEYGD433LD' # your FourSquare Access Token

VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 2P0SWWQDIBFLA5CAKFSN00DTUMGCDXS5HDODYCPDGF5PKAOU
CLIENT_SECRET:0F45RJR2B5A12QM4D1RBPGMPJPQ4MLIMKUFGWDAIPIWS4YSB


In [16]:
import requests
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']

        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood Title', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

# type your answer here
toronto_venues = getNearbyVenues(
    names = toronto['Neighborhood'],
    latitudes = toronto['Latitude'],
    longitudes = toronto['Longitude'],
    radius = 500)

Regent Park, Harbourfront
Queen's Park, Ontario Provincial Government
Garden District, Ryerson
St. James Town
The Beaches
Berczy Park
Central Bay Street
Christie
Richmond, Adelaide, King
Dufferin, Dovercourt Village
Harbourfront East, Union Station, Toronto Islands
Little Portugal, Trinity
The Danforth West, Riverdale
Toronto Dominion Centre, Design Exchange
Brockton, Parkdale Village, Exhibition Place
India Bazaar, The Beaches West
Commerce Court, Victoria Hotel
Studio District
Lawrence Park
Roselawn
Davisville North
Forest Hill North & West, Forest Hill Road Park
High Park, The Junction South
North Toronto West, Lawrence Park
The Annex, North Midtown, Yorkville
Parkdale, Roncesvalles
Davisville
University of Toronto, Harbord
Runnymede, Swansea
Moore Park, Summerhill East
Kensington Market, Chinatown, Grange Park
Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport
R

In [17]:
print(toronto_venues.shape)
toronto_venues.head()

(1595, 7)


,Neighborhood Title,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Regent Park, Harbourfront",43.65426,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
1,"Regent Park, Harbourfront",43.65426,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
2,"Regent Park, Harbourfront",43.65426,-79.360636,Cooper Koo Family YMCA,43.653249,-79.358008,Distribution Center
3,"Regent Park, Harbourfront",43.65426,-79.360636,Body Blitz Spa East,43.654735,-79.359874,Spa
4,"Regent Park, Harbourfront",43.65426,-79.360636,Impact Kitchen,43.656369,-79.356980,Restaurant


In [18]:
print(f"There are {toronto_venues['Venue Category'].nunique()} unique categories.")

There are 232 unique categories.


In [19]:
toronto_venues[toronto_venues['Venue Category'] == 'Neighborhood']

,Neighborhood Title,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
261,The Beaches,43.676357,-79.293031,Upper Beaches,43.680563,-79.292869,Neighborhood
405,"Richmond, Adelaide, King",43.650571,-79.384568,Downtown Toronto,43.653232,-79.385296,Neighborhood
506,"Harbourfront East, Union Station, Toronto Islands",43.640816,-79.381752,Harbourfront,43.639526,-79.380688,Neighborhood
939,Studio District,43.659526,-79.340923,Leslieville,43.662070,-79.337856,Neighborhood


In [20]:
# one hot encoding
toronto_onehot = pd.get_dummies(
    toronto_venues[['Venue Category']],
    prefix = "",
    prefix_sep = "")

print(toronto_onehot.shape)

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood Title'] = toronto_venues['Neighborhood Title']

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]
toronto_onehot.set_index('Neighborhood Title', inplace = True)

(1595, 232)


In [21]:
toronto_grouped = toronto_onehot.groupby('Neighborhood Title').mean().reset_index()
toronto_grouped.head()

,Neighborhood Title,Adult Boutique,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Tibetan Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Yoga Studio
0,Berczy Park,0.0,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.017857,0.0,0.0,0.000000,0.0,0.000000
1,"Brockton, Parkdale Village, Exhibition Place",0.0,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000
2,"Business reply mail Processing Centre, South C...",0.0,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000
3,"CN Tower, King and Spadina, Railway Lands, Har...",0.0,0.0625,0.0625,0.0625,0.125,0.1875,0.125,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000
4,Central Bay Street,0.0,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.016129,0.0,0.0,0.016129,0.0,0.016129


In [22]:
toronto_grouped.shape

(39, 233)

In [23]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [24]:
import numpy as np
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood Title']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood Title'] = toronto_grouped['Neighborhood Title']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.rename(columns = {'Neighborhood Title' : 'Neighborhood'}, inplace = True)
neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Coffee Shop,Cocktail Bar,Beer Bar,Farmers Market,Cheese Shop,Bakery,Seafood Restaurant,Restaurant,Liquor Store,Breakfast Spot
1,"Brockton, Parkdale Village, Exhibition Place",Café,Coffee Shop,Breakfast Spot,Grocery Store,Italian Restaurant,Pet Store,Performing Arts Venue,Restaurant,Climbing Gym,Nightclub
2,"Business reply mail Processing Centre, South C...",Garden,Auto Workshop,Skate Park,Farmers Market,Spa,Burrito Place,Light Rail Station,Restaurant,Brewery,Garden Center
3,"CN Tower, King and Spadina, Railway Lands, Har...",Airport Service,Airport Lounge,Airport Terminal,Harbor / Marina,Airport,Coffee Shop,Rental Car Location,Sculpture Garden,Boat or Ferry,Plane
4,Central Bay Street,Coffee Shop,Italian Restaurant,Café,Sandwich Place,Salad Place,Burger Joint,Bubble Tea Shop,Chinese Restaurant,Business Service,Discount Store


In [25]:
from sklearn.cluster import KMeans

# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood Title', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 0], dtype=int32)

In [26]:
toronto_grouped_clustering.head()

,Adult Boutique,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,...,Tibetan Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Yoga Studio
0,0.0,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.017857,0.0,0.0,0.000000,0.0,0.000000
1,0.0,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000
2,0.0,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000
3,0.0,0.0625,0.0625,0.0625,0.125,0.1875,0.125,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000
4,0.0,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.016129,0.0,0.0,0.016129,0.0,0.016129


In [27]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = toronto

# merge manhattan_grouped with manhattan_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head() # check the last columns!

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,1,Coffee Shop,Bakery,Park,Pub,Restaurant,Theater,Café,Breakfast Spot,Electronics Store,Event Space
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,1,Coffee Shop,Sushi Restaurant,Yoga Studio,Park,Smoothie Shop,Burrito Place,Café,Sandwich Place,College Auditorium,College Cafeteria
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937,1,Clothing Store,Coffee Shop,Café,Cosmetics Shop,Middle Eastern Restaurant,Bubble Tea Shop,Japanese Restaurant,Lingerie Store,Electronics Store,Pizza Place
3,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418,1,Coffee Shop,Café,Gastropub,Cocktail Bar,American Restaurant,Seafood Restaurant,Clothing Store,Restaurant,Creperie,Department Store
4,M4E,East Toronto,The Beaches,43.676357,-79.293031,4,Neighborhood,Health Food Store,Trail,Pub,Museum,Martial Arts School,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant


In [28]:
import matplotlib.cm as cm
import matplotlib.colors as colors

# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [29]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
18,Central Toronto,0,Park,Bus Line,Business Service,Swim School,Adult Boutique,Movie Theater,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant
20,Central Toronto,0,Pizza Place,Breakfast Spot,Hotel,Food & Drink Shop,Department Store,Park,Sandwich Place,Gym,Trail,Moroccan Restaurant
21,Central Toronto,0,Mexican Restaurant,Jewelry Store,Trail,Sushi Restaurant,Adult Boutique,Movie Theater,Market,Martial Arts School,Mediterranean Restaurant,Men's Store
33,Downtown Toronto,0,Park,Playground,Trail,Adult Boutique,Movie Theater,Market,Martial Arts School,Mediterranean Restaurant,Men's Store,Mexican Restaurant


### View the DataFrame of all Clusters

In [30]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Downtown Toronto,1,Coffee Shop,Bakery,Park,Pub,Restaurant,Theater,Café,Breakfast Spot,Electronics Store,Event Space
1,Downtown Toronto,1,Coffee Shop,Sushi Restaurant,Yoga Studio,Park,Smoothie Shop,Burrito Place,Café,Sandwich Place,College Auditorium,College Cafeteria
2,Downtown Toronto,1,Clothing Store,Coffee Shop,Café,Cosmetics Shop,Middle Eastern Restaurant,Bubble Tea Shop,Japanese Restaurant,Lingerie Store,Electronics Store,Pizza Place
3,Downtown Toronto,1,Coffee Shop,Café,Gastropub,Cocktail Bar,American Restaurant,Seafood Restaurant,Clothing Store,Restaurant,Creperie,Department Store
5,Downtown Toronto,1,Coffee Shop,Cocktail Bar,Beer Bar,Farmers Market,Cheese Shop,Bakery,Seafood Restaurant,Restaurant,Liquor Store,Breakfast Spot
6,Downtown Toronto,1,Coffee Shop,Italian Restaurant,Café,Sandwich Place,Salad Place,Burger Joint,Bubble Tea Shop,Chinese Restaurant,Business Service,Discount Store
7,Downtown Toronto,1,Grocery Store,Café,Park,Restaurant,Italian Restaurant,Baby Store,Candy Store,Nightclub,Athletics & Sports,Coffee Shop
8,Downtown Toronto,1,Coffee Shop,Café,Restaurant,Bakery,Clothing Store,Gym,Deli / Bodega,Thai Restaurant,Concert Hall,Bookstore
9,West Toronto,1,Bakery,Pharmacy,Grocery Store,Liquor Store,Middle Eastern Restaurant,Music Venue,Park,Coffee Shop,Café,Brewery
10,Downtown Toronto,1,Coffee Shop,Aquarium,Hotel,Café,Fried Chicken Joint,Restaurant,Scenic Lookout,Italian Restaurant,Brewery,Music Venue


In [31]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
29,Central Toronto,2,Playground,Restaurant,Adult Boutique,Museum,Martial Arts School,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop


In [32]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
19,Central Toronto,3,Home Service,Garden,Fast Food Restaurant,Adult Boutique,Movie Theater,Market,Martial Arts School,Mediterranean Restaurant,Men's Store,Mexican Restaurant


In [33]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,East Toronto,4,Neighborhood,Health Food Store,Trail,Pub,Museum,Martial Arts School,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant
